# Calculating Hotel Occupancy

---

> Hotel occupancy is a critical factor during the booking process and can provide additional insight into the likelihood of cancellations and/or forecasting future ADR.
> 
> However, *there's no clear indication of the total number of guest rooms for either hotel.*
>
> 
> I will determine the maximum number of rooms occupied for each date for each hotel, which can be used as a placeholder max occupancy number.

---

# Import Packages and Read Data

In [21]:
## Used to upload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
## Enabling access to custom functions in separate directory

# # Import necessary modules
# import os
# import sys

# # Construct the absolute path to the 'src' directory
# src_path = os.path.abspath(os.path.join('../..', 'src'))

# # Append the path to 'sys.path'
# if src_path not in sys.path:
#     sys.path.append(src_path)

# import db_utils, eda

## Data Handling
import datetime as dt
import pandas as pd
import numpy as np
import sweetviz as sv

## Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 50)
%matplotlib inline

## Load Pre-Reviewed Data

In [23]:
data_path = '../../data/3.1_temporally_updated_data.feather'
df_data = pd.read_feather(data_path)
df_data

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,DepartureDate,Length of Stay,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,ArrivalWeek,ArrivalDay,ArrivalDateDayName,ArrivalDateMonthName
0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,873e7749-dcb5-4c01-b54f-46082557421a,2015-07-01,2015-07-01,0,2014-07-24,342,45,21,45,21,22,44,27,3,Wednesday,July
1,0,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6,NULL,0,Transient,80.00,0,0,Check-Out,H2,34ffe79b-1c75-4a7e-9353-974435aeaa76,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July
2,0,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,be96389e-f204-4118-beb0-f8c57752beb0,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July
3,0,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,1f50ec6f-ea8d-48b1-86cb-aa4999b8d460,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July
4,0,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,d38621e9-377a-4302-a587-bd1267eee0ac,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,2,0.00,0,HB,GBR,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,241,NULL,0,Transient,207.03,0,1,Check-Out,H1,e8240939-e608-4177-8b0a-9c1acbdc25b6,2017-08-31,2017-09-07,7,2017-05-15,115,35,16,28,23,26,14,35,4,Thursday,August
119386,0,2,1.00,0,HB,ITA,Online TA,TA/TO,0,0,0,G,G,3,No Deposit,240,NULL,0,Transient,312.29,1,1,Check-Out,H1,55c827d5-b681-4b84-be50-9315d15bb979,2017-08-31,2017-09-07,7,2017-02-18,201,35,16,28,23,55,48,35,4,Thursday,August
119387,1,2,0.00,0,HB,ESP,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,207.00,0,2,Canceled,H1,858a4520-54e8-44ee-a36b-6e69d2a06988,2017-08-31,2017-09-03,3,2017-08-14,0,35,16,32,19,1,60,35,4,Thursday,August
119388,0,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,40,NULL,0,Contract,114.80,0,0,Check-Out,H1,37a6b3f1-27e8-4719-ae03-a4d280730517,2017-08-31,2017-09-07,7,2017-02-21,198,35,16,28,23,52,51,35,4,Thursday,August


# Calculate Daily Occupancies

# Hotel 1

In [24]:
df_data['HotelNumber'].unique()

array(['H1', 'H2'], dtype=object)

In [25]:
hotel = 'H1'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h1 = df_data[hotel_filter]

df_data_h1

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,DepartureDate,Length of Stay,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,ArrivalWeek,ArrivalDay,ArrivalDateDayName,ArrivalDateMonthName
0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,873e7749-dcb5-4c01-b54f-46082557421a,2015-07-01,2015-07-01,0,2014-07-24,342,45,21,45,21,22,44,27,3,Wednesday,July
56,0,2,1.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,69.60,0,1,Check-Out,H1,31c5d140-b059-49d0-95fd-3e74c8e09250,2015-07-01,2015-07-08,7,2015-02-02,156,45,21,38,28,60,32,27,3,Wednesday,July
57,0,2,0.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,55.68,0,0,Check-Out,H1,699abce4-6762-4c20-95a9-d97f1a7e4bb0,2015-07-01,2015-07-08,7,2015-02-02,156,45,21,38,28,60,32,27,3,Wednesday,July
58,0,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,115,NULL,0,Contract,82.35,0,0,Check-Out,H1,18d33e9f-b3fe-4008-98cd-17403ba2e36d,2015-07-01,2015-07-08,7,2015-02-06,152,45,21,38,28,56,36,27,3,Wednesday,July
59,0,3,0.00,0,BB,ESP,Offline TA/TO,TA/TO,0,0,0,A,C,2,No Deposit,6,NULL,0,Transient,79.59,1,1,Check-Out,H1,d7a9ab38-3b65-4030-affd-0d3f7c8aac86,2015-07-01,2015-07-06,5,2015-03-13,115,45,21,40,26,21,71,27,3,Wednesday,July
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119384,1,2,0.00,0,BB,ESP,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,240,NULL,0,Transient,196.67,0,1,Canceled,H1,e19790e2-91f4-4dbb-bf15-a1b5d35c47a5,2017-08-31,2017-09-03,3,2017-06-22,39,35,16,32,19,54,7,35,4,Thursday,August
119385,0,2,0.00,0,HB,GBR,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,241,NULL,0,Transient,207.03,0,1,Check-Out,H1,e8240939-e608-4177-8b0a-9c1acbdc25b6,2017-08-31,2017-09-07,7,2017-05-15,115,35,16,28,23,26,14,35,4,Thursday,August
119386,0,2,1.00,0,HB,ITA,Online TA,TA/TO,0,0,0,G,G,3,No Deposit,240,NULL,0,Transient,312.29,1,1,Check-Out,H1,55c827d5-b681-4b84-be50-9315d15bb979,2017-08-31,2017-09-07,7,2017-02-18,201,35,16,28,23,55,48,35,4,Thursday,August
119387,1,2,0.00,0,HB,ESP,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,207.00,0,2,Canceled,H1,858a4520-54e8-44ee-a36b-6e69d2a06988,2017-08-31,2017-09-03,3,2017-08-14,0,35,16,32,19,1,60,35,4,Thursday,August


In [26]:
min_date = df_data_h1['ArrivalDate'].min()
max_date = df_data_h1['DepartureDate'].max()

print(f'The earliest date is: {min_date}.\nThe latest date is: {max_date}.')

The earliest date is: 2015-07-01 00:00:00.
The latest date is: 2017-09-14 00:00:00.


### ChatGPT-Generated Code

#### Walkthrough of Code Below

Certainly! Let's break down the corrected code into its key steps, explaining each part and its purpose in calculating the number of active hotel reservations for each date:

**3. Generate Counts for Arrivals and Departures**

```python
arrivals = df_data_h1['ArrivalDate'].value_counts().rename('count')
departures = df_data_h1['DepartureDate'].value_counts().rename('count')
```
- Counts how many reservations start (`arrivals`) and end (`departures`) on each date. The `value_counts()` method tallies occurrences of each date, and `rename('count')` changes the Series name to `'count'`, which aids in clarity for later operations.

**4. Combine Arrival and Departure Counts**

```python
df_counts = pd.concat([arrivals, -departures]).sort_index().reset_index()
df_counts.columns = ['Date', 'Count']
```
- Combines the arrivals and departures into a single DataFrame, `df_counts`, with arrivals contributing positively to the count and departures negatively (indicating the end of a reservation). The data is then sorted by date.

**5. Aggregate Counts on the Same Date**

```python
df_counts = df_counts.groupby('Date').sum()
```
- Since the combination of arrivals and departures could result in multiple entries for the same date, this step aggregates (sums) all counts for each date. This ensures each date is unique, addressing the initial issue of duplicate labels.

**6. Generate a Complete Date Range and Reindex**

```python
date_range = pd.date_range(start=df_counts.index.min(), end=df_counts.index.max())
df_counts = df_counts.reindex(date_range, fill_value=0)
```
- Creates a continuous range of dates covering the entire period from the earliest to the latest date in `df_counts`. It then reindexes `df_counts` to include every date in this range, filling any dates without data with `0`, ensuring there's a record for every single day in the period.

**7. Calculate Cumulative Sum for Active Reservations**

```python
df_counts['Active_Reservations'] = df_counts['Count'].cumsum()
```
- Computes the cumulative sum of the daily net reservation counts (`Count`). This step effectively calculates the total number of active reservations for each date by adding up the arrivals and subtracting the departures as they occur over time.

**Conclusion**

- The final output, `df_counts['Active_Reservations']`, shows the total number of active reservations for each date in the range. This method is efficient and avoids the problem of duplicate labels by ensuring that each date is unique before the reindexing step, leveraging pandas' capabilities for handling time series data.

### Code

In [27]:
def get_counts(dataframe, arrivaldate, departuredate, name = 'count'):
    
    '''Generate counts for arrivals and departures on their respective dates.'''
    
    arrivals = (dataframe[arrivaldate]
                .value_counts()
                .rename(name))
    
    departures = (dataframe[departuredate]
                  .value_counts()
                  .rename(name))
    
    return arrivals, departures


def aggregate_counts_by_date(arrivals, departures):
    
    '''Create a DataFrame from arrivals and departures,
    marking departures as negative.'''
    
    df_counts = (pd.concat([arrivals, -departures])
                 .sort_index()
                 .reset_index())
    
    df_counts.columns = ['Date', 'Count']
    
    return df_counts.groupby('Date').sum()


def generate_date_range(df_counts):
    '''Generate a complete date range covering the period in df_data.'''
    
    return pd.date_range(start=df_counts.index.min(),
                         end=df_counts.index.max())


def reindex_and_fill_zero(df_counts, date_range):
    '''Reindex the aggregated count DataFrame to include all dates in the range,
    filling missing dates with 0'''
    
    return df_counts.reindex(date_range, fill_value=0)


def calculate_daily_active_res(df_counts):
    '''Calculate the cumulative sum to determine active reservations for each date'''
   
    return df_counts['Count'].cumsum()


def calculate_daily_occupancy(dataframe, arrivaldate, departuredate, name = 'count'):
    
    arrivals, departures = get_counts(dataframe, arrivaldate, departuredate, name = 'count')
    
    daily_counts = aggregate_counts_by_date(arrivals, departures)
    
    date_range = generate_date_range((daily_counts))
    
    df_reindexed = reindex_and_fill_zero(daily_counts, date_range)
    
    return calculate_daily_active_res(df_reindexed)

In [28]:
df_counts = calculate_daily_occupancy(df_data_h1, 'ArrivalDate', 'DepartureDate')
df_counts.name = 'Active_Reservations'
df_counts

2015-07-01     41
2015-07-02     78
2015-07-03    108
2015-07-04    147
2015-07-05    154
             ... 
2017-09-10      4
2017-09-11      3
2017-09-12      2
2017-09-13      2
2017-09-14      0
Freq: D, Name: Active_Reservations, Length: 807, dtype: int64

In [29]:
# # Generate counts for arrivals and departures on their respective dates
# arrivals = df_data_h1['ArrivalDate'].value_counts().rename('count')
# departures = df_data_h1['DepartureDate'].value_counts().rename('count')
# arrivals.head(), departures.head()

In [30]:
# # Create a DataFrame from arrivals and departures, marking departures as negative
# df_counts = pd.concat([arrivals, -departures]).sort_index().reset_index()
# df_counts.columns = ['Date', 'Count']
# df_counts

In [31]:
# # Aggregate counts on the same date to avoid duplicate labels
# df_counts = df_counts.groupby('Date').sum()
# df_counts

In [32]:
# # Generate a complete date range covering the period in df_data
# date_range = pd.date_range(start=df_counts.index.min(),
#                            end=df_counts.index.max())
# date_range

In [33]:
# # Reindex the aggregated count DataFrame to include all dates in the range, filling missing dates with 0
# df_counts = df_counts.reindex(date_range, fill_value=0)
# df_counts

In [34]:
# # Calculate the cumulative sum to determine active reservations for each date
# df_counts['Active_Reservations'] = df_counts['Count'].cumsum()

# df_counts['Active_Reservations']

#### Groupby.Sum vs. Cumsum

The use of both `groupby().sum()` and the `cumsum()` methods serves two different purposes in the process of calculating the total number of active reservations for each date. Here's a clarification of the roles each step plays in the computation:

**1. GroupBy().sum()**

- **Purpose:** This step aggregates the daily net changes in reservations (arrivals and departures) for each unique date. Since arrivals are counted positively and departures negatively, the sum for each date tells us the net reservation change on that day. 
- **What It Solves:** If, for instance, 5 reservations start (arrive) and 3 end (depart) on a particular date, the net change in reservations for that day would be +2. This calculation consolidates all changes into a single value per date, ensuring there's no duplication of dates in the dataset, which is necessary for the next steps.

**2. cumsum()**

- **Purpose:** The cumulative sum (`cumsum()`) takes these daily net changes and accumulates them over the entire period to calculate the total number of active reservations for each date. It essentially adds up the net changes from the start date, rolling forward, to show how many reservations are active on any given day.
- **What It Solves:** This step provides the running total of active reservations. It accounts for the ongoing balance of reservations as they begin and end over time, showing the total active reservations on each date. This is crucial for understanding the capacity or occupancy on any given day.

**Illustrative Example:**

Let's say you have data for three days:

- **Day 1:** 5 arrivals, 0 departures (net +5)
- **Day 2:** 3 arrivals, 1 departure (net +2)
- **Day 3:** 2 arrivals, 4 departures (net -2)

After `groupby().sum()`, you'd have a net change sequence of [+5, +2, -2].

Applying `cumsum()` to this sequence gives you the total active reservations for each day: [5, 7, 5]. This demonstrates how the occupancy evolves:

- **Day 1:** Starts with 5,
- **Day 2:** Increases to 7,
- **Day 3:** Decreases back to 5.

**Conclusion:**

- **`groupby().sum()`** is used for condensing the dataset into a form where each date has a single net change value, resolving any issues with duplicate dates.
- **`cumsum()`** transforms these net changes into a running total of active reservations, reflecting how the number of active reservations builds up or reduces over time.

## Adding Arrival/Departure Occupancies to Original Data

In [35]:
df_counts.head(10)

2015-07-01     41
2015-07-02     78
2015-07-03    108
2015-07-04    147
2015-07-05    154
2015-07-06    172
2015-07-07    190
2015-07-08    201
2015-07-09    213
2015-07-10    222
Freq: D, Name: Active_Reservations, dtype: int64

In [36]:
# Define active_reservations
active_reservations = df_counts

# Find the maximum occupancy
max_occupancy = active_reservations.max()

# Map the occupancy on arrival and departure dates to each reservation
df_data_h1.loc[:, 'occupancy_at_arrival'] = df_data_h1.loc[:, 'ArrivalDate'].map(active_reservations)
df_data_h1.loc[:, 'occupancy_at_departure'] = df_data_h1.loc[:, 'DepartureDate'].map(active_reservations)

# Convert these occupancies to percentages of the maximum occupancy
df_data_h1.loc[:, 'occupancy_pct_at_arrival'] = (df_data_h1.loc[:, 'occupancy_at_arrival'] / max_occupancy)
df_data_h1.loc[:, 'occupancy_pct_at_departure'] = (df_data_h1.loc[:, 'occupancy_at_departure'] / max_occupancy)

df_data_h1

C:\Users\Ben\AppData\Local\Temp\ipykernel_16008\2332825845.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h1.loc[:, 'occupancy_at_arrival'] = df_data_h1.loc[:, 'ArrivalDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_16008\2332825845.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h1.loc[:, 'occupancy_at_departure'] = df_data_h1.loc[:, 'DepartureDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_16008\2332825845.py:12: SettingWithCopyW

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,DepartureDate,Length of Stay,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,ArrivalWeek,ArrivalDay,ArrivalDateDayName,ArrivalDateMonthName,occupancy_at_arrival,occupancy_at_departure,occupancy_pct_at_arrival,occupancy_pct_at_departure
0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,873e7749-dcb5-4c01-b54f-46082557421a,2015-07-01,2015-07-01,0,2014-07-24,342,45,21,45,21,22,44,27,3,Wednesday,July,41,41,0.12,0.12
56,0,2,1.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,69.60,0,1,Check-Out,H1,31c5d140-b059-49d0-95fd-3e74c8e09250,2015-07-01,2015-07-08,7,2015-02-02,156,45,21,38,28,60,32,27,3,Wednesday,July,41,201,0.12,0.56
57,0,2,0.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,55.68,0,0,Check-Out,H1,699abce4-6762-4c20-95a9-d97f1a7e4bb0,2015-07-01,2015-07-08,7,2015-02-02,156,45,21,38,28,60,32,27,3,Wednesday,July,41,201,0.12,0.56
58,0,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,115,NULL,0,Contract,82.35,0,0,Check-Out,H1,18d33e9f-b3fe-4008-98cd-17403ba2e36d,2015-07-01,2015-07-08,7,2015-02-06,152,45,21,38,28,56,36,27,3,Wednesday,July,41,201,0.12,0.56
59,0,3,0.00,0,BB,ESP,Offline TA/TO,TA/TO,0,0,0,A,C,2,No Deposit,6,NULL,0,Transient,79.59,1,1,Check-Out,H1,d7a9ab38-3b65-4030-affd-0d3f7c8aac86,2015-07-01,2015-07-06,5,2015-03-13,115,45,21,40,26,21,71,27,3,Wednesday,July,41,172,0.12,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119384,1,2,0.00,0,BB,ESP,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,240,NULL,0,Transient,196.67,0,1,Canceled,H1,e19790e2-91f4-4dbb-bf15-a1b5d35c47a5,2017-08-31,2017-09-03,3,2017-06-22,39,35,16,32,19,54,7,35,4,Thursday,August,271,102,0.76,0.29
119385,0,2,0.00,0,HB,GBR,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,241,NULL,0,Transient,207.03,0,1,Check-Out,H1,e8240939-e608-4177-8b0a-9c1acbdc25b6,2017-08-31,2017-09-07,7,2017-05-15,115,35,16,28,23,26,14,35,4,Thursday,August,271,22,0.76,0.06
119386,0,2,1.00,0,HB,ITA,Online TA,TA/TO,0,0,0,G,G,3,No Deposit,240,NULL,0,Transient,312.29,1,1,Check-Out,H1,55c827d5-b681-4b84-be50-9315d15bb979,2017-08-31,2017-09-07,7,2017-02-18,201,35,16,28,23,55,48,35,4,Thursday,August,271,22,0.76,0.06
119387,1,2,0.00,0,HB,ESP,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,207.00,0,2,Canceled,H1,858a4520-54e8-44ee-a36b-6e69d2a06988,2017-08-31,2017-09-03,3,2017-08-14,0,35,16,32,19,1,60,35,4,Thursday,August,271,102,0.76,0.29


In [37]:
df_data_h1

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,DepartureDate,Length of Stay,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,ArrivalWeek,ArrivalDay,ArrivalDateDayName,ArrivalDateMonthName,occupancy_at_arrival,occupancy_at_departure,occupancy_pct_at_arrival,occupancy_pct_at_departure
0,0,2,0.00,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NULL,NULL,0,Transient,0.00,0,0,Check-Out,H1,873e7749-dcb5-4c01-b54f-46082557421a,2015-07-01,2015-07-01,0,2014-07-24,342,45,21,45,21,22,44,27,3,Wednesday,July,41,41,0.12,0.12
56,0,2,1.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,69.60,0,1,Check-Out,H1,31c5d140-b059-49d0-95fd-3e74c8e09250,2015-07-01,2015-07-08,7,2015-02-02,156,45,21,38,28,60,32,27,3,Wednesday,July,41,201,0.12,0.56
57,0,2,0.00,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,156,NULL,0,Contract,55.68,0,0,Check-Out,H1,699abce4-6762-4c20-95a9-d97f1a7e4bb0,2015-07-01,2015-07-08,7,2015-02-02,156,45,21,38,28,60,32,27,3,Wednesday,July,41,201,0.12,0.56
58,0,2,0.00,0,HB,GBR,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,115,NULL,0,Contract,82.35,0,0,Check-Out,H1,18d33e9f-b3fe-4008-98cd-17403ba2e36d,2015-07-01,2015-07-08,7,2015-02-06,152,45,21,38,28,56,36,27,3,Wednesday,July,41,201,0.12,0.56
59,0,3,0.00,0,BB,ESP,Offline TA/TO,TA/TO,0,0,0,A,C,2,No Deposit,6,NULL,0,Transient,79.59,1,1,Check-Out,H1,d7a9ab38-3b65-4030-affd-0d3f7c8aac86,2015-07-01,2015-07-06,5,2015-03-13,115,45,21,40,26,21,71,27,3,Wednesday,July,41,172,0.12,0.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119384,1,2,0.00,0,BB,ESP,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,240,NULL,0,Transient,196.67,0,1,Canceled,H1,e19790e2-91f4-4dbb-bf15-a1b5d35c47a5,2017-08-31,2017-09-03,3,2017-06-22,39,35,16,32,19,54,7,35,4,Thursday,August,271,102,0.76,0.29
119385,0,2,0.00,0,HB,GBR,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,241,NULL,0,Transient,207.03,0,1,Check-Out,H1,e8240939-e608-4177-8b0a-9c1acbdc25b6,2017-08-31,2017-09-07,7,2017-05-15,115,35,16,28,23,26,14,35,4,Thursday,August,271,22,0.76,0.06
119386,0,2,1.00,0,HB,ITA,Online TA,TA/TO,0,0,0,G,G,3,No Deposit,240,NULL,0,Transient,312.29,1,1,Check-Out,H1,55c827d5-b681-4b84-be50-9315d15bb979,2017-08-31,2017-09-07,7,2017-02-18,201,35,16,28,23,55,48,35,4,Thursday,August,271,22,0.76,0.06
119387,1,2,0.00,0,HB,ESP,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,NULL,0,Transient,207.00,0,2,Canceled,H1,858a4520-54e8-44ee-a36b-6e69d2a06988,2017-08-31,2017-09-03,3,2017-08-14,0,35,16,32,19,1,60,35,4,Thursday,August,271,102,0.76,0.29


# Hotel 2

In [38]:
hotel = 'H2'

hotel_filter = (df_data['HotelNumber'] == hotel)

df_data_h2 = df_data[hotel_filter]

df_counts_h2 = calculate_daily_occupancy(df_data_h2, 'ArrivalDate', 'DepartureDate')
df_counts_h2.name = 'Active_Reservations'
df_counts_h2

2015-07-01     79
2015-07-02    128
2015-07-03     71
2015-07-04     68
2015-07-05     57
             ... 
2017-09-07      2
2017-09-08      2
2017-09-09      1
2017-09-10      1
2017-09-11      0
Freq: D, Name: Active_Reservations, Length: 804, dtype: int64

In [39]:
# Define active_reservations
active_reservations = df_counts

# Find the maximum occupancy
max_occupancy = active_reservations.max()

# Map the occupancy on arrival and departure dates to each reservation
df_data_h2.loc[:, 'occupancy_at_arrival'] = df_data_h2.loc[:, 'ArrivalDate'].map(active_reservations)
df_data_h2.loc[:, 'occupancy_at_departure'] = df_data_h2.loc[:, 'DepartureDate'].map(active_reservations)

# Convert these occupancies to percentages of the maximum occupancy
df_data_h2.loc[:, 'occupancy_pct_at_arrival'] = (df_data_h2.loc[:, 'occupancy_at_arrival'] / max_occupancy)
df_data_h2.loc[:, 'occupancy_pct_at_departure'] = (df_data_h2.loc[:, 'occupancy_at_departure'] / max_occupancy)

df_data_h2

C:\Users\Ben\AppData\Local\Temp\ipykernel_16008\184567899.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h2.loc[:, 'occupancy_at_arrival'] = df_data_h2.loc[:, 'ArrivalDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_16008\184567899.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_h2.loc[:, 'occupancy_at_departure'] = df_data_h2.loc[:, 'DepartureDate'].map(active_reservations)
C:\Users\Ben\AppData\Local\Temp\ipykernel_16008\184567899.py:12: SettingWithCopyWarn

,IsCanceled,Adults,Children,Babies,Meal,Country,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,HotelNumber,UUID,ArrivalDate,DepartureDate,Length of Stay,BookingDate,DaysSinceBooking,ArrivalDate_DaysBeforeHoliday,ArrivalDate_DaysAfterHoliday,DepartureDate_DaysBeforeHoliday,DepartureDate_DaysAfterHoliday,BookingDate_DaysBeforeHoliday,BookingDate_DaysAfterHoliday,ArrivalWeek,ArrivalDay,ArrivalDateDayName,ArrivalDateMonthName,occupancy_at_arrival,occupancy_at_departure,occupancy_pct_at_arrival,occupancy_pct_at_departure
1,0,1,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,6,NULL,0,Transient,80.00,0,0,Check-Out,H2,34ffe79b-1c75-4a7e-9353-974435aeaa76,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July,41,108,0.12,0.30
2,0,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,be96389e-f204-4118-beb0-f8c57752beb0,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July,41,108,0.12,0.30
3,0,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,1f50ec6f-ea8d-48b1-86cb-aa4999b8d460,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July,41,108,0.12,0.30
4,0,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,d38621e9-377a-4302-a587-bd1267eee0ac,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July,41,108,0.12,0.30
5,0,2,0.00,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6,NULL,0,Transient,101.50,0,0,Check-Out,H2,193a3a70-9544-4771-b172-31dc9eabcbfd,2015-07-01,2015-07-03,2,2014-10-17,259,45,21,43,23,52,63,27,3,Wednesday,July,41,108,0.12,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119362,0,2,0.00,0,SC,FRA,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,132.67,0,1,Check-Out,H2,7265f386-5dcd-42bf-9962-2dfccaf1132a,2017-08-31,2017-09-03,3,2017-08-27,7,35,16,32,19,39,12,35,4,Thursday,August,271,102,0.76,0.29
119366,0,2,0.00,0,SC,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9,NULL,0,Transient,99.00,0,0,Check-Out,H2,43d3775b-3feb-4f90-8597-ab92922dc8a7,2017-08-31,2017-09-02,2,2017-02-14,200,35,16,33,18,59,44,35,4,Thursday,August,271,154,0.76,0.43
119372,0,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,D,F,4,No Deposit,9,NULL,0,Transient,170.80,0,2,Check-Out,H2,6f7d97d4-ab6d-4df7-8aab-b7d671f55310,2017-08-31,2017-09-02,2,2016-10-03,334,35,16,33,18,2,49,35,4,Thursday,August,271,154,0.76,0.43
119380,0,2,0.00,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,89,NULL,0,Transient,104.40,0,0,Check-Out,H2,2665afe4-8c01-43b7-a2db-80122963f613,2017-08-31,2017-09-07,7,2017-05-14,116,35,16,28,23,27,13,35,4,Thursday,August,271,22,0.76,0.06


In [40]:
full_dataset = pd.concat([df_data_h1, df_data_h2], axis = 0)

full_dataset.to_feather('../../data/3.2_data_with_occupancies.feather', compression = 'zstd')